In [1]:
import numpy as np
import pandas as pd

 - **Layers**: an Integer value representing the total number of hidden layers in the network (input and output layers are extra).
 
 - **Nodes**: an integer array of size [0,..,Layers+1] containing the dimensions of the neural
network. Nodes[0] shall represent the input size (typically, 50), Nodes[Layers+1]
shall represent the number of output nodes (typically, 1). All other values Nodes[i]
represent the number of nodes in hidden layer i.

 - **NNodes**: a possible alternative to the Nodes parameter for situations where you want
each hidden layer of the neural network to be of the same size. In this case, the size of
the output layer is assumed to be 1, and the size of the input layer can be inferred from
the dataset.

 - **Activations**: an array of size [0,..,Layers+1] (for the sake of compatibility) in which
Activations[0] and Activations[Layers+1] are not used, while all other
Activations[i] values are labels indicating the activation function used in layer i.
This allows you to build neural networks with different activation functions in each layer.

 - **ActivationFn**: a possible alternative to Activations when all hidden layers of your neural
network use the same activation function.

In [3]:
class NeuralNetwork:
    #Layers: an Integer value representing the total number of hidden layers in the network 
    #        (input and output layers are extra)

    def __init__(self, Layers, Nodes, NNodes, Activations, ActivationFn):
        self.Layers = Layers
        self.Nodes = Nodes
        self.NNodes = NNodes
        self.Activations = Activations
        self.ActivationFn = ActivationFn
        
    def Relu(e):
        return max(0,e)
    
    def leakyRelu(e,a=0.1):
        if e > 0:
            return e
        else:
            return a*e
        
    def sigmoid(e):
        return 1/(1+np.exp(1)**-e)
    
    def tanh(e):
        return 2*sigmoid(2*e) - 1
    
    def applyActivation(layer,i):
        acttype = Activations[i]
        if acttype == "Relu":
            return layer.applymap(Relu)
        elif acttype == "leaky":
            return layer.applymap(leakyRelu)
        elif acttype == "sigmoid":
            return layer.applymap(sigmoid)
        elif acttype == "tanh":
            return layer.applymap(tanh)
    
    def loss(z,y):
        # Performs L2 loss (for this project)
        L = 0.5*(z-y)**2 # Assumes the squaring is element wise
        L = sum(L) * (1/len(z)) # Take average of all the losses
        return L
    
    def initialize_weights():
        weights = []
        for i in range(len(self.Nodes)-1):
            M = self.Nodes[i] + 1  #+1 for bias term
            N = self.Nodes[i+1]  
            w = np.random.normal(loc=0,scale = np.sqrt(2/(M+N)),size=(M,N)) 
            weights.append(w)
        return weights
        
        
    def forward_pass(self, X, y, weights):
        # Assume X already has a column of ones for bias term.
        # Assume weights include the weights for the bias term when going into next layer
        
        savings = [X]
        
        # From input layer to first hidden layer
        h = X.dot(weights[0].T) # Get first hidden layer without the bias node added in
        h['ones'] = 1 # Add in bias node to the hidden layer
        savings.append(h) # Saving intermediate values
        hact = applyActivation(h,0) # Perform activation
        hact['ones'] = 1
        savings.append(hact) # Saving intermediate values
        h = hact
     
        for i in range(1,len(weights)-1):
            if i != len(weights)-1: # A hidden layer
                h = h.dot(weights[i])
                h['ones'] = 1 # Add in bias node to the hidden layer
                savings.append(h) # Saving intermediate values
                hact = applyActivation(h,i) # Perform activation
                hact['ones'] = 1
                savings.append(hact) # Saving intermediate values
                h = hact
            else: # For Z value/vector
                z = h.dot(weights[i])
                savings.append(z)
                
                # Calculate loss
                L = loss(z,y)
                savings.append(L) # Are we saving average loss over the batch?
                
        return savings

In [6]:
iris = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

In [7]:
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [8]:
def ReLu(e):
    return max(0,e)

In [9]:
def leakyRelu(e,a=0.1):
    if e > 0:
        return e
    else:
        return a*e

In [10]:
def sigmoid(e):
    return 1/(1+np.exp(1)**-e)

In [11]:
def tanh(e):
    return 2*sigmoid(2*e) - 1

In [12]:
df = iris.drop('species',axis=1)
# b = petal_width

In [13]:
w = np.random.normal(loc=0,scale = np.sqrt(2/(3+3)),size=(3,4))
w

array([[-0.2469103 ,  0.3094092 ,  0.90890716,  0.55163971],
       [-0.20161322, -0.84930151,  0.00476418, -0.80020967],
       [ 0.87599308,  0.94693851,  0.14293612, -0.76119476]])

w = np.array(df.mean())
w = [list(w)]*3
w = np.array(w)
w

In [14]:
one = np.array(df.iloc[0])
one[len(one)-1] = 1
one

array([5.1, 3.5, 1.4, 1. ])

In [15]:
h = w.dot(one)
h

array([ 1.64779942, -4.7943225 ,  7.22076531])

In [16]:
pd.Series(h).apply(tanh)

0    0.928555
1   -0.999863
2    0.999999
dtype: float64

In [17]:
h = pd.Series(h).apply(leakyRelu,args=(0.2,))
h

0    1.647799
1   -0.958864
2    7.220765
dtype: float64

In [18]:
b2 = 2
w2 = np.array([1,2,3,b2])
z = w2.dot(np.append(h,1))

In [19]:
z

23.392366348425707

In [20]:
df["petal_width"] = 1

In [21]:
df

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,1
1,4.9,3.0,1.4,1
2,4.7,3.2,1.3,1
3,4.6,3.1,1.5,1
4,5.0,3.6,1.4,1
...,...,...,...,...
145,6.7,3.0,5.2,1
146,6.3,2.5,5.0,1
147,6.5,3.0,5.2,1
148,6.2,3.4,5.4,1


In [22]:
w

array([[-0.2469103 ,  0.3094092 ,  0.90890716,  0.55163971],
       [-0.20161322, -0.84930151,  0.00476418, -0.80020967],
       [ 0.87599308,  0.94693851,  0.14293612, -0.76119476]])

In [23]:
w.T

array([[-0.2469103 , -0.20161322,  0.87599308],
       [ 0.3094092 , -0.84930151,  0.94693851],
       [ 0.90890716,  0.00476418,  0.14293612],
       [ 0.55163971, -0.80020967, -0.76119476]])

In [24]:
h = df.dot(w.T).applymap(ReLu)
#df.dot(w.T)
h

,0,1,2
0,1.647799,0,7.220765
1,1.542477,0,6.572097
2,1.562850,0,6.571993
3,1.738382,0,6.418287
4,1.703431,0,7.227860
...,...,...,...
145,4.551886,0,8.692042
146,4.314164,0,7.839589
147,4.601268,0,8.516844
148,4.980886,0,8.661408


In [29]:
h["ones"] = 1

In [30]:
h

,0,1,2,ones
0,0,2.121289,0.000000,1
1,0,2.045728,0.000000,1
2,0,1.970496,0.000000,1
3,0,1.878945,0.000000,1
4,0,2.071516,0.000000,1
...,...,...,...,...
145,0,1.956001,1.004284,1
146,0,1.836375,1.298243,1
147,0,1.863307,0.950407,1
148,0,1.661929,0.563435,1


In [31]:
z = h.dot(w2)

In [32]:
# Average Loss over batch
Lb = 0.5*((z-iris["petal_width"])**2)
(1/len(Lb))*np.sum(Lb)

21.118146926688247